In [1]:
!pwd

/home/studio-lab-user/Blackbox-Coresets-VI/psvi/notebooks


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import os 
import sys
sys.path.append("/home/studio-lab-user/Blackbox-Coresets-VI")

In [4]:
method_args = {
    'data_folder': '/home/studio-lab-user/all_data/vi_data',
    'test_ratio': 0.2
}

In [5]:
from psvi.experiments.experiments_utils import read_dataset
from psvi.models.frequentist_models import FreqLogReg, FreqOneLayer
from torch.utils.data import DataLoader
from psvi.models.frequentist_models import RunFrequentistModel
from psvi.inference.baselines import run_el2n_coreset


import torch
import torch.optim as optim
import torch.distributions as dist
import torch.nn.functional as F



In [6]:
x, y, xt, yt, N, D, \
train_dataset, test_dataset, num_classes = read_dataset("webspam", method_args)

X is either low-dimensional or not very sparse, so converting to a numpy array
X is either low-dimensional or not very sparse, so converting to a numpy array


In [7]:
train_dataset

In [9]:
#criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCEWithLogitsLoss()
model = FreqLogReg(input_dim=D, num_classes=num_classes)
optimizer = optim.SGD(model.parameters(), lr=1e-3)


In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [11]:
# train an epoch
for batch_idx, (data, target) in enumerate(train_dataloader):
    #if args['cuda']:
        #data, target = data.cuda(), target.cuda()
    #Variables in Pytorch are differenciable. 
    #data, target = Variable(data), Variable(target)
    #This will zero out the gradients for this batch. 
    optimizer.zero_grad()
    output = model(data)
    # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
    loss = criterion(output.squeeze(), target)
    #dloss/dx for every Variable 
    loss.backward()
    #to do a one-step update on our parameter.
    optimizer.step()


In [34]:
target.long()

tensor([1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1])

In [14]:
output.size()

torch.Size([41, 2])

In [15]:
target.size()

torch.Size([41])

In [34]:
softmax_fn = torch.nn.Softmax()
sigmoid_fn = torch.nn.Sigmoid()
#outputs_prob = softmax_fn(output.squeeze())
#outputs_prob

In [38]:
softmax_fn(torch.tensor(-300.0))
sigmoid_fn(torch.tensor(1))

/tmp/ipykernel_126/2327056272.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_fn(torch.tensor(-300.0))


tensor(0.7311)

In [32]:
outputs_prob.unsqueeze(1).size()

torch.Size([41, 1])

In [23]:
outputs_prob_0 = 1. - outputs_prob

In [33]:
torch.hstack((outputs_prob.unsqueeze(1), outputs_prob_0.unsqueeze(1))).size()

torch.Size([41, 2])

In [27]:
_, predicted = torch.max(output.data, 1)

In [29]:
(predicted == target.long()).sum().item()

26

In [18]:
output.squeeze().gt(0.5).float().eq(target).sum()

tensor(24)

In [20]:
dist.Bernoulli(probs=outputs_prob).log_prob(target).sum()

tensor(-92.8361, grad_fn=<SumBackward0>)

In [55]:
target.size(0)

41

In [10]:
criterion(output, torch.tensor(1))

ValueError: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.

In [19]:
torch.tensor(1).unsqueeze(0).size()

torch.Size([1])

In [37]:
#F.one_hot(target.long(), num_classes=2)


In [16]:
x, y, xt, yt, N, D, \
train_dataset, test_dataset, num_classes = read_dataset("webspam", method_args)


X is either low-dimensional or not very sparse, so converting to a numpy array
X is either low-dimensional or not very sparse, so converting to a numpy array


In [7]:
len(train_dataset)

126185

In [23]:
model = FreqLogReg(input_dim=D, num_classes=num_classes)

In [39]:
run_freq = RunFrequentistModel(
    train_dataset=train_dataset, test_dataset=test_dataset, model=model, 
    is_logreg=True, num_epochs=20
)

run_freq.train()

Epoch: 0
Accuracy: 0.7437289357185364, NLL: 0.5029181241989136
Epoch: 10
Accuracy: 0.7688131928443909, NLL: 0.481990247964859


In [40]:
#for param in model.parameters():
#  print(param.data)


In [41]:
el2n_scores = run_freq.get_el2n_scores()

In [42]:
run_freq.get_largest_el2n_indices(10)

[46691, 519, 82201, 119540, 16419, 72864, 116814, 84964, 98355, 51911]

In [43]:
el2n_scores[0:20]

tensor([0.8149, 0.8843, 0.5626, 0.2827, 0.4713, 0.3625, 0.2463, 0.2712, 0.5595,
        0.3034, 0.3195, 0.2402, 0.2956, 0.8847, 0.3406, 0.4722, 0.5980, 0.3927,
        0.1587, 0.3212])

In [7]:
run_el2n_coreset(
    x=x,
    y=y,
    xt=xt,
    yt=yt,
    mc_samples=4,
    num_epochs=30,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    log_every=10,
    N=N,
    D=D,
    seed=0,
    mcmc=False,
    lr0net=1e-3,  # initial learning rate for optimizer
    nc=num_classes,
    data_minibatch=128,
) 

Epoch: 0
Accuracy: 0.6068795919418335, NLL: 0.6582239866256714
Epoch: 10
Accuracy: 0.641906201839447, NLL: 0.5972860455513


  3%|▎         | 1/30 [00:03<01:33,  3.22s/it]

predictive accuracy: 60.70%


 37%|███▋      | 11/30 [00:06<00:10,  1.85it/s]

predictive accuracy: 60.67%


100%|██████████| 30/30 [00:09<00:00,  3.03it/s]

predictive accuracy: 35.96%


{'accs': [0.6069939136505127, 0.6066510677337646, 0.3595794439315796],
 'nlls': [0.7019806504249573, 0.8734530806541443, 1.7704914808273315],
 'csizes': [0, 10, 20],
 'times': [],
 'wt_index': [{},
  {41186: 12618.5,
   119540: 12618.5,
   72864: 12618.5,
   1534: 12618.5,
   98355: 12618.5,
   60817: 12618.5,
   46691: 12618.5,
   119029: 12618.5,
   45438: 12618.5,
   63387: 12618.5},
  {41186: 6309.25,
   119540: 6309.25,
   72864: 6309.25,
   1534: 6309.25,
   98355: 6309.25,
   60817: 6309.25,
   46691: 6309.25,
   119029: 6309.25,
   45438: 6309.25,
   63387: 6309.25,
   18205: 6309.25,
   118535: 6309.25,
   58446: 6309.25,
   50390: 6309.25,
   107960: 6309.25,
   99605: 6309.25,
   8913: 6309.25,
   96515: 6309.25,
   43456: 6309.25,
   87349: 6309.25}]}